In [1]:
import pandas
import numpy as np
import import_ipynb
import FeatureStore

importing Jupyter notebook from FeatureStore.ipynb


In [2]:
from sklearn.preprocessing import StandardScaler

#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
def scaleDataFrame(dataframe, debugPrint = False):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe)
    dataframe = pandas.DataFrame(scaled_features, index=dataframe.index, columns=dataframe.columns)
    
    #check
    if debugPrint == True:
        for column in dataframe:
            print("[%s] mean = %.4f, variance = %.4f" % 
                  (column, dataframe[column].mean(), dataframe[column].var()))
    
    return dataframe, scaler.mean_, scaler.scale_

In [3]:
results = {}
Period = [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]
Alpha = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
Layers = [(100,100), (100,100,100)]
Activation = ['relu', 'logistic', 'tanh']
# fill results with initial zero values
for period in Period:
    results[period] = {}
    for alpha in Alpha:
        results[period][alpha] = {}
        for activation in Activation:
            results[period][alpha][activation] = {}
            for layer in Layers:
                 results[period][alpha][activation][layer] = 0.0

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score

from datetime import datetime


#Alpha = [0.1]; Period = [1]
#C = {1.0}
#grid = {'C': np.power(10.0, np.arange(-4, 0))}

sigma1 = 0
mu1 = 0
clf1 = 0
accs = {}


            
for period in Period:
    X_train = FeatureStore.getTrainDataset(period)
    y_train = FeatureStore.getTrainTarget(period)
    
    X_train, sigma, mu = scaleDataFrame(X_train, debugPrint = False)
    #print(sigma)
    #print(mu)

    for alpha in Alpha:
        for activation in Activation:
            for layer in Layers:
                print("trying Alpha = %f, period = %d, function = %s, layer config %s" 
                      % (alpha, period, activation, layer))
                kf = KFold(n_splits=3, shuffle=False, random_state=241)

                clf = MLPClassifier(hidden_layer_sizes = layer, activation=activation, solver='sgd', 
                                    alpha = alpha, max_iter = 100000, random_state=241)
                if (period == 40):
                    sigma1 = sigma
                    mu1 = mu
                    clf1 = clf
                #N = 800
                #XX = X_train.iloc[:N, :]
                #yy = y_train[:N]
                XX = X_train
                yy = y_train
                #print(X_train.shape)
                #print(y_train.shape)
                #print("from", datetime.now())
                scores = cross_val_score(clf, XX, yy, scoring='accuracy', cv=kf)

                #print("spent iterations %d" % (clf.n_iter_))
                #print(clf)
                #print(vars(clf))
                #print(clf.n_iter_)
                #print(scores)
                print("accuracy=", scores.mean())
                results[period][alpha][activation] = scores.mean() 
                #print("to", datetime.now())
                #accs[period].append(scores.mean())
                if period in accs and not (accs[period] is None):
                    accs[period].append(scores.mean())
                else:
                    accs[period] = [scores.mean()]
        
for k in accs:
    accs[k] = np.max(accs[k])

#print(accs)

trying Alpha = 1.000000, period = 0, function = relu, layer config (100, 100)
accuracy= 0.7842974718859989
trying Alpha = 1.000000, period = 0, function = relu, layer config (100, 100, 100)
accuracy= 0.7861940939136406
trying Alpha = 1.000000, period = 0, function = logistic, layer config (100, 100)
accuracy= 0.5591064683663833
trying Alpha = 1.000000, period = 0, function = logistic, layer config (100, 100, 100)
accuracy= 0.5591064683663833
trying Alpha = 1.000000, period = 0, function = tanh, layer config (100, 100)
accuracy= 0.7890269551034423
trying Alpha = 1.000000, period = 0, function = tanh, layer config (100, 100, 100)
accuracy= 0.7842947892522963
trying Alpha = 1.000000, period = 0, function = relu, layer config (100, 100)
accuracy= 0.7842974718859989
trying Alpha = 1.000000, period = 0, function = relu, layer config (100, 100, 100)


In [ ]:
for k, v in accs.items():
    print("%d;%f" % (k, v))
    

print(clf1, sigma1, mu1)
print(results)

import json
with open('resultDNN.json', 'w') as fp:
    json.dump(results, fp)

In [ ]:
from sklearn.neural_network import MLPClassifier

def backtestStrategy():
    #trying Alpha = 0.100000, period = 40, function = relu
    period = 40
    penalty = 0.1
    activation = 'relu'
    
    X_train = FeatureStore.getTrainDataset(period)
    y_train = FeatureStore.getTrainTarget(period)
    
    clf = MLPClassifier(hidden_layer_sizes=(100, 100), activation=activation, solver='sgd', 
                    alpha = penalty, max_iter = 100000, random_state=241)
    clf.fit(X_train, y_train)
    
    X_test  = FeatureStore.getTestDataset(period)
    y_test  = FeatureStore.getTestTarget(period)
    print(X_test.shape)
    print(y_test.shape)
    print(clf.score(X_test, y_test))
    
    pass

backtestStrategy()